## Use of Linear and Logsitic Regression Coefficients with L1 Lasso (L1) and Ridge (L2) Regularization for Feature Selection in Machine Learning

-------------

## Linear Regression

### Basic Assumption

- Linear relationship
- Feature space X should have gaussian distrbution
- Features are not correlated with other
- Features are in same scale i.e. have same variance

###### Regularization is a technique to discourage the complexity of the model. It does this by penalizing the loss function. This Helps to solve the overfitting problem

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.metrics import accuracy_score

In [5]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [6]:
titanic = sns.load_dataset('titanic')

In [7]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [8]:
titanic.drop(labels = ['age','deck'], axis = 1, inplace = True)

In [9]:
titanic = titanic.dropna()

In [11]:
titanic.isnull().sum()

survived       0
pclass         0
sex            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [12]:
titanic = titanic.dropna()

In [13]:
titanic.isnull().sum()

survived       0
pclass         0
sex            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [14]:
data = titanic[['pclass','sex','sibsp','parch','embarked','who','alone']].copy()

In [15]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,male,1,0,S,man,False
1,1,female,1,0,C,woman,False
2,3,female,0,0,S,woman,True
3,1,female,1,0,S,woman,False
4,3,male,0,0,S,man,True


In [16]:
data.isnull().sum()

pclass      0
sex         0
sibsp       0
parch       0
embarked    0
who         0
alone       0
dtype: int64

In [17]:
sex = {'male': 0, 'female':1}
data['sex'] = data['sex'].map(sex)

In [18]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,S,man,False
1,1,1,1,0,C,woman,False
2,3,1,0,0,S,woman,True
3,1,1,1,0,S,woman,False
4,3,0,0,0,S,man,True


In [19]:
ports = {'S':0, 'C':1, 'Q':2}
data['embarked'] = data['embarked'].map(ports)

In [20]:
who = {'man':0, 'woman':1, 'child':2}
data['who'] = data['who'].map(who)

In [21]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,0,False
1,1,1,1,0,1,1,False
2,3,1,0,0,0,1,True
3,1,1,1,0,0,1,False
4,3,0,0,0,0,0,True


In [22]:
X = data.copy()
y = titanic['survived']

In [23]:
X.shape, y.shape

((889, 7), (889,))

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 43)

## Estimation of Coefficients of Linear Regression

In [25]:
sel = SelectFromModel(LinearRegression())

In [26]:
sel.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression())

In [27]:
sel.get_support()

array([ True,  True, False, False, False,  True, False])

In [28]:
sel.estimator_.coef_

array([-0.13750402,  0.26606466, -0.07470416, -0.0668525 ,  0.04793674,
        0.23857799, -0.12929595])

In [29]:
mean = np.mean(np.abs(sel.estimator_.coef_))

In [30]:
mean

0.13727657291370784

In [31]:
np.abs(sel.estimator_.coef_)

array([0.13750402, 0.26606466, 0.07470416, 0.0668525 , 0.04793674,
       0.23857799, 0.12929595])

In [32]:
features = X_train.columns[sel.get_support()]
features

Index(['pclass', 'sex', 'who'], dtype='object')

In [33]:
X_train_reg = sel.transform(X_train)
X_test_reg = sel.transform(X_test)

In [34]:
X_test_reg.shape

(267, 3)

In [35]:
def run_RandomForest(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test, y_pred))

In [37]:
%%time
run_RandomForest(X_train_reg, X_test_reg, y_train, y_test)

Accuracy:  0.8239700374531835
Wall time: 687 ms


In [38]:
%%time
run_RandomForest(X_train, X_test, y_train, y_test)

Accuracy:  0.8239700374531835
Wall time: 571 ms


## Logistic Regression Coefficient with L1 Regularization

In [44]:
sel = SelectFromModel(LogisticRegression(penalty = 'l1', C = 0.05, solver ='liblinear' ))
sel.fit(X_train, y_train)
sel.get_support()

array([ True,  True,  True, False, False,  True, False])

In [45]:
sel.estimator_.coef_

array([[-0.54044793,  0.78022031, -0.14085259,  0.        ,  0.        ,
         0.94123335,  0.        ]])

In [46]:
X_train_l1 = sel.transform(X_train)
X_test_l1 = sel.transform(X_test)

In [47]:
%%time
run_RandomForest(X_train_l1, X_test_l1, y_train, y_test)

Accuracy:  0.8277153558052435
Wall time: 543 ms


## L2 Regularization

In [49]:
sel = SelectFromModel(LogisticRegression(penalty = 'l2', C = 0.05, solver ='liblinear' ))
sel.fit(X_train, y_train)
sel.get_support()

sel.estimator_.coef_

X_train_l2 = sel.transform(X_train)
X_test_l2 = sel.transform(X_test)



In [50]:
%%time
run_RandomForest(X_train_l2, X_test_l2, y_train, y_test)

Accuracy:  0.8239700374531835
Wall time: 465 ms
